# Домашнее  задание  № 1.  
*Графематический и морфологический анализ текста*

Вариант D

**Задание:**

Вычислить и проанализировать статистику морфологической омонимии в РЯ, взяв для этого 2-3 текста разных стилей/жанров
и составив программу, которая на базе выбранного, программно подключаемого морфопроцессора определяет:

– общее количество словоупотреблений, число различных словоформ, количество уникальных лемм, число незнакомых слов,
коэффициент лексического богатства текста (= отношение числа различных лемм к общему числу словоупотреблений);

–  абсолютную и относительную (с учетом/без учета неизменяемых слов) частоту омонимичных словоформ, абсолютную и
относительную частоту словоформ с лексико-морфологической омонимией, максимальное и среднее число омонимов у 
словоформ текста, словоформы с наибольшим числом омонимов, наиболее частотный омоним

В качестве подключаемого морфопроцессора можно взять, например:
  mystem(http://api.yandex.ru/mystem/downloads/) или АОТ (http://www.aot.ru )


**Решение:**

Для проведения анализа были выбраны 3 текста разных жанров (научный, художественный и публицистический). Тексты приведены в файле texts.py

В качестве подключаемого морфопроцессора я выбрала Mystem и pymorphy2 (https://pymorphy2.readthedocs.org/en/latest/#). 

В первом случае выбор обоснован интересом к продукции Яндекса, второй - более широким функционалом. Поэтому часть заданий реализовано с помощью Mystem, а другая часть с помощью Pymorph.

**Подготовка данных:**
Прежде чем начинать анализировать данные, нужно их "почистить", привести в удобный для работы вид. Изначально мы имеем тесты со знаками препинания, цифрами, пробелами и другими символами. Нас интересуют лишь слова, поэтому была написана функция *make_clear_text*. Рассмотрим ее работу подробнее:

In [1]:
# подключение необходимых библиотек
import pymorphy2
from texts import *
from pymystem3 import Mystem

In [2]:
def make_clear_text(text):
    # текст без спец символов/цифр/пунктуации
    m = Mystem()
    clear_text = []
    # переводим все заглавные буквы в строчные
    lm = m.analyze(text.lower())
    for i in range(0, len(lm)):
        # в этом месте мы отбрасываем все посторонные символы, остается массив слов
        if 'analysis' in lm[i]:
            clear_text.append(lm[i]['text'])
    return clear_text

def no_one_symbol_text(text):
    # убираем слова состоящие из одного символа
    for i in text:
        if len(i) == 1:
            text.remove(i)
    return text

Для написания этой функции был использован морфологический процессор Mystem. Он анализирует все элементы текста, расставляя тег "analysis" только словам, игнорируя все остальные символы. Этим я и воспользовалась. Хотя изначально была идея использовать регулярные выражения. 
Теперь все готово для проведения анализа, начнем с **общего количества совоупотреблений**.

In [3]:
def usage_count(text):
    # общее количество словоупотреблений
    # приводим данные в удобный вид, считаем, сколько слов в нашем тексте
    result = len(make_clear_text(text))
    print("Общее количество словоупотреблений: ", result)
    return result

Далее реализована функция, которая считает **количество различных словоформ**. Для этого необходимо сделать множество словоупотреблений, таким образом мы уберем повторяющиеся элементы.

In [4]:
def count_different_word_forms(text):
    # число различных словоформ
    # приводим данные в удобный вид, получаем список всех слов в тексте
    clear_text = make_clear_text(text)
    # находим множество всех слов в тексте (теперь нет повторяющихся слов) и посчитаем их количество
    result = len(set(clear_text))
    print("Число различных словоформ: ", result)

Теперь реализуем функцию подсчитывающую **количество уникальных лемм**. Здесь я использовала морфопроцессор Mystem чтобы найти леммы. 

In [5]:
def number_of_unique_lemmas(text):
    # количество уникальных лемм
    m = Mystem()
    # текст с леммами
    lemmas = m.lemmatize(text)
    # делим текст на слова
    lemmatized_text = ''.join(lemmas)
    # чистим данные от лишних символов
    clear_text = no_one_symbol_text(make_clear_text(lemmatized_text))
    result = len(set(clear_text))
    print("Количество уникальных лемм: ", result)
    return result

Следующим пунктом найдем **количество неизвестных слов**. У морфопроцессора **pymorph** есть очень удобная функция **word_is_known(word)**, она возвращает **True**, если слово известно, иначе - **False**. 

In [6]:
def number_of_unknown_words(text):
    # число незнакомых слов
    morph = pymorphy2.MorphAnalyzer()
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    # суммируем количество неизвестных слов
    result = sum([not morph.word_is_known(x) for x in clear_text])
    print("Число незнакомых слов: ", result)

Далее рассмотрим метод вычисления коэффициента **лексического богатства текста**. Здесь очень просто, потому что мы уже умеем считать количество уникальных лемм и общее количество словоупотреблений. Осталось только разделить их друг на друга.

In [7]:
def lexical_richness_of_the_text_index(text):
    # коэффициент лексического богатства текста
    result = number_of_unique_lemmas(text)/usage_count(text)
    print("Коэффициент лексического богатства текста:", result)

Рассмотрим функцию **абсолютной и относительной частоты омонимичных словоформ** без учета неизменяемых слов. Здесь нам нужно посчитать все слова у которых парсер выдает более одного типа разбора + надо учесть и отбросить те омонимы, у которых лексемы одинаковые. Для этого мы берем очередной омоним и склоняем все его разборы, чтобы найти лексемы. Далее находим множества этих лексем, и если оказывается, что в таких множествах 1 элемент - такой омоним неизменяемый, мы его отбрасываем. Но такой способ не всегда срабатывает. Существуют сокращениях слов (например "руб" - рубль), "руб" является неизменяемым омонимом, но когда мы начнем его склонять, морфопроцессор посчитает, что это "рубль", и склонять мы будем уже совсем не то, что хотели. Поэтому будем использовать встроенный в морфопроцессор тег m.tag если в нем есть параметр Fixd, то это неизменяемое слово. Опытным путем подтвердилось, что такой тег гарантирует работу для сокращений.

In [8]:
def frequency_homonymous_word_forms(text):
    # абсолютная и относительная (без учета/с учетом неизменяемых слов) частота омонимичных словоформ
    morph = pymorphy2.MorphAnalyzer()
    result = 0
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    for j in clear_text:
        m = morph.parse(j)
        for k in range(0, len(m)):
            lexeme = []
            unchangeable_tag = []
            unchangeable_tag.append('Fixd' in m[k].tag)
            unchangeable = []
            lex = m[k].lexeme
            for i in lex:
                lexeme.append(i.word)
            lexeme = set(lexeme)
            unchangeable.append(len(lexeme) == 1)
        unchangeable = unchangeable + unchangeable_tag
        if len(m) > 1 and True not in unchangeable:
            result += 1
    print("Абсолютная частота омонимичных словоформ без учета неизменяемых слов: ", result)
    print("Относительная частота омонимичных словоформ без учета неизменяемых слов: ", result/len(clear_text))

Функция **абсолютной и относительной частоты омонимичных словоформ** c учетом неизменяемых слов. С учетом неизменяемых слов все эти действия просто не выполняем, нас интересуют как неизменяемые так и изменяемые слова.

In [9]:
def frequency_homonymous_word_forms_full(text):
    # абсолютная и относительная (с учетом неизменяемых слов) частота омонимичных словоформ
    morph = pymorphy2.MorphAnalyzer()
    result = 0
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    for j in clear_text:
        m = morph.parse(j)
        # set_of_normal_forms = set([m[i].normal_form for i in range(0, len(m))])
        # print(set_of_normal_forms)
        if len(m) > 1:
            result += 1
    print("Абсолютная частота омонимичных словоформ с учетом неизменяемых слов: ", result)
    print("Относительная частота омонимичных словоформ с учетом неизменяемых слов: ", result/len(clear_text))


In [10]:
def frequency_of_word_forms_with_lexicalmorphological_homonymy(text):
    # абсолютная и относительнаю частота словоформ с лексико-морфологической омонимией
    morph = pymorphy2.MorphAnalyzer()
    result = 0
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    set_of_lexem = []
    for j in clear_text:
        m = morph.parse(j)
        if morph.word_is_known(j):
            set_of_lexem = set([m[i][4][0][2] for i in range(0, len(m))])
        if len(m) > 1 and len(set_of_lexem) > 1:
            result += 1
    print("Абсолютная частота словоформ с лексико-морфологической омонимией: ", result)
    print("Относительная частота словоформ с лексико-морфологической омонимией: ", result/len(clear_text))

Теперь рассмотрим функцию, которая находит **максимальное и среднее число омонимов у словоформ текста, словоформу с наибольшим числом омонимов, наиболее частотный омоним**.

Для того чтобы найти максимальное число омонимов нужно запоминать текущее число омонимов у очередного слова и сравнимать с предыдущим максимальным числом, если текущее больше чем предыдущее, сделаем его максимальным и так далее. Перебрав таким образом все слова, надем максимум.

Чтобы найти среднее нужно просто суммировать количество омонимов каждого слова потом разделить на их количество.

Словоформа с наибольшим числом омонимов находится как и максимальное число омонимов: мы просто каждый раз запоминаем слово вместе с числом.

С наиболее частотным омонимом дела обстоят сложнее, тут нужно хранить словарик, у которого ключи это слова, а значения ключей - число, означающее сколько раз нам такое слово встретилось. Если слово нам еще не встречалось, мы добавляем в словарь новый ключ, если слово оказалось омонимом, то значение этого ключа увеличивается на единицу. В конце обработки текста находим наибольшее значение и показываем как результат его ключ те самый популярый омоним. 

In [11]:
def maximum_and_average_number_of_homonyms_in_the_text_word_forms(text):
    # максимальное и среднее число омонимов у словоформ текста, словоформа с наибольшим числом омонимов,
    # наиболее частотный омоним
    morph = pymorphy2.MorphAnalyzer()
    result = {}
    max_homonym = 0
    mean_homonym = 0
    counter = 0
    clear_text = no_one_symbol_text(make_clear_text(text))
    for j in clear_text:
        set_of_lexem = []
        if j not in result:
            result[j] = 0
        m = morph.parse(j)
        if len(m) > 1:
            result[j] += 1
            counter += 1
            mean_homonym += len(m)
            if len(m) > max_homonym:
                max_homonym = len(m)
                wordform_with_max_homonym = j
    print("Максимальное число омонимов у словоформ: ", max_homonym)
    print("Среднее число омонимов у словоформ: ", mean_homonym/counter)
    print("Словоформа с наибольшим числом омонимов: ", wordform_with_max_homonym)
    print("Наиболее частотный омоним: ",  max(result, key=result.get))

**Анализ текстов:**

Теперь применим все написанные функции к текстам

Научный текст:

In [13]:
count_different_word_forms(scientific_text)
lexical_richness_of_the_text_index(scientific_text)
number_of_unknown_words(scientific_text)
frequency_homonymous_word_forms_full(scientific_text)
frequency_homonymous_word_forms(scientific_text)
frequency_of_word_forms_with_lexicalmorphological_homonymy(scientific_text)
maximum_and_average_number_of_homonyms_in_the_text_word_forms(scientific_text)

Число различных словоформ:  78
Количество уникальных лемм:  71
Общее количество словоупотреблений:  84
Коэффициент лексического богатства текста: 0.8452380952380952
Число незнакомых слов:  2
Абсолютная частота омонимичных словоформ с учетом неизменяемых слов:  51
Относительная частота омонимичных словоформ с учетом неизменяемых слов:  0.6710526315789473
Абсолютная частота омонимичных словоформ без учета неизменяемых слов:  44
Относительная частота омонимичных словоформ без учета неизменяемых слов:  0.5789473684210527
Абсолютная частота словоформ с лексико-морфологической омонимией:  20
Относительная частота словоформ с лексико-морфологической омонимией:  0.2631578947368421
Максимальное число омонимов у словоформ:  26
Среднее число омонимов у словоформ:  3.736842105263158
Словоформа с наибольшим числом омонимов:  их
Наиболее частотный омоним:  на


Художественный текст:

In [14]:
count_different_word_forms(artistic_text)
lexical_richness_of_the_text_index(artistic_text)
number_of_unknown_words(artistic_text)
frequency_homonymous_word_forms_full(artistic_text)
frequency_homonymous_word_forms(artistic_text)
frequency_of_word_forms_with_lexicalmorphological_homonymy(artistic_text)
maximum_and_average_number_of_homonyms_in_the_text_word_forms(artistic_text)

Число различных словоформ:  304
Количество уникальных лемм:  262
Общее количество словоупотреблений:  397
Коэффициент лексического богатства текста: 0.6599496221662469
Число незнакомых слов:  10
Абсолютная частота омонимичных словоформ с учетом неизменяемых слов:  174
Относительная частота омонимичных словоформ с учетом неизменяемых слов:  0.5838926174496645
Абсолютная частота омонимичных словоформ без учета неизменяемых слов:  147
Относительная частота омонимичных словоформ без учета неизменяемых слов:  0.49328859060402686
Абсолютная частота словоформ с лексико-морфологической омонимией:  72
Относительная частота словоформ с лексико-морфологической омонимией:  0.24161073825503357
Максимальное число омонимов у словоформ:  28
Среднее число омонимов у словоформ:  3.365740740740741
Словоформа с наибольшим числом омонимов:  его
Наиболее частотный омоним:  новые


Публицистический текст

In [12]:
count_different_word_forms(artistic_text)
lexical_richness_of_the_text_index(artistic_text)
number_of_unknown_words(artistic_text)
frequency_homonymous_word_forms_full(artistic_text)
frequency_homonymous_word_forms(artistic_text)
frequency_of_word_forms_with_lexicalmorphological_homonymy(artistic_text)
maximum_and_average_number_of_homonyms_in_the_text_word_forms(artistic_text)

Число различных словоформ:  304
Количество уникальных лемм:  262
Общее количество словоупотреблений:  397
Коэффициент лексического богатства текста: 0.6599496221662469
Число незнакомых слов:  10
Абсолютная частота омонимичных словоформ с учетом неизменяемых слов:  174
Относительная частота омонимичных словоформ с учетом неизменяемых слов:  0.5838926174496645
Абсолютная частота омонимичных словоформ без учета неизменяемых слов:  147
Относительная частота омонимичных словоформ без учета неизменяемых слов:  0.49328859060402686
Абсолютная частота словоформ с лексико-морфологической омонимией:  73
Относительная частота словоформ с лексико-морфологической омонимией:  0.24496644295302014
Максимальное число омонимов у словоформ:  28
Среднее число омонимов у словоформ:  3.365740740740741
Словоформа с наибольшим числом омонимов:  его
Наиболее частотный омоним:  новые
